# VacationPy
----
We have used the final weather data placed in Final_city_data.csv to plan future vacations.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store results into DataFrame
* Loading the csv exported from weather data to a DataFrame

In [13]:
filepath = "Final_city_data.csv"
city_data = pd.DataFrame(pd.read_csv(filepath))
city_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564 entries, 0 to 563
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        564 non-null    object 
 1   Lat         564 non-null    float64
 2   Lng         564 non-null    float64
 3   Max Temp    564 non-null    float64
 4   Humidity    564 non-null    int64  
 5   Cloudiness  564 non-null    int64  
 6   Wind Speed  564 non-null    float64
 7   Country     560 non-null    object 
 8   Date        564 non-null    int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 35.3+ KB


### Humidity Heatmap
* Configuring gmaps used for creating heat maps
* We have stored Lat and Lng as locations and Humidity as the weight to create the heat map
* Adding the Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
city_data = city_data.dropna()
# Store 'Lat' and 'Lng' into  locations 
locations = city_data[["Lat", "Lng"]]

# Humidity

Humidity_data = city_data["Humidity"]


In [5]:
# Creating humidity Heatmap layer
fig = gmaps.figure(center=(35,135),zoom_level=3)

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity_data, 
                                 dissipating=False, max_intensity=200,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Creating new DataFrame fitting weather criteria for vacations.
* Narrow down the cities to fit weather conditions like max Temp inbetween 70 and 80, Wind Speed is less than 20 and Cloudiness is equal to zero.
* Droping any rows will null values.

In [14]:
city_data = city_data.dropna()
city_data = city_data.loc[city_data['Max Temp']>70,:]

city_data = city_data.loc[city_data['Wind Speed']<20,:]
city_data = city_data.loc[city_data['Cloudiness']==0,:]

city_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97 entries, 0 to 558
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        97 non-null     object 
 1   Lat         97 non-null     float64
 2   Lng         97 non-null     float64
 3   Max Temp    97 non-null     float64
 4   Humidity    97 non-null     int64  
 5   Cloudiness  97 non-null     int64  
 6   Wind Speed  97 non-null     float64
 7   Country     97 non-null     object 
 8   Date        97 non-null     int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 6.8+ KB


### Hotel Map
* Adding a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
city_data['Hotel Name'] = ""
city_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Richards Bay,-28.78,32.04,284.20,69,0,1.54,ZA,1594954455,
2,Sesheke,-17.48,24.30,278.15,64,0,0.50,ZM,1594954456,
3,Hermanus,-34.42,19.23,279.82,89,0,1.34,ZA,1594954457,
5,Assiut,27.18,31.18,298.15,61,0,5.70,EG,1594954460,
8,Bredasdorp,-34.53,20.04,280.15,87,0,3.10,ZA,1594954463,


In [16]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
params = {
    "location" : "",
    "type": "lodging",
    "radius": "5000",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in city_data.iterrows():

    # get LAt and Lng from df
    Lat = row['Lat']
    Lng = row['Lng']

    # add keyword to params dict
    params["location"] = f"{Lat},{Lng}"
    
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    
    results = response['results']
   
    try:
        print(f"Closest Hotel is {results[0]['name']}.")   
        city_data.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0.
Closest Hotel is BON Hotel Waterfront Richards Bay.
------------
Retrieving Results for Index 2.
Closest Hotel is Protea Hotel by Marriott Zambezi River Lodge.
------------
Retrieving Results for Index 3.
Closest Hotel is Misty Waves Boutique Hotel.
------------
Retrieving Results for Index 5.
Closest Hotel is Florence Hotel Assiut.
------------
Retrieving Results for Index 8.
Closest Hotel is Bredasdorp Country Manor.
------------
Retrieving Results for Index 13.
Closest Hotel is Hotel Serra da Chela.
------------
Retrieving Results for Index 25.
Closest Hotel is Alrwasi Hotel.
------------
Retrieving Results for Index 34.
Missing field/result... skipping.
------------
Retrieving Results for Index 40.
Closest Hotel is Nuevo Hotel Plaza.
------------
Retrieving Results for Index 49.
Missing field/result... skipping.
------------
Retrieving Results for Index 59.
Closest Hotel is Nebitchi Hotel.
------------
Retrieving Results for Index 63.
Closest Hotel i

In [17]:
city_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Richards Bay,-28.78,32.04,284.20,69,0,1.54,ZA,1594954455,BON Hotel Waterfront Richards Bay
2,Sesheke,-17.48,24.30,278.15,64,0,0.50,ZM,1594954456,Protea Hotel by Marriott Zambezi River Lodge
3,Hermanus,-34.42,19.23,279.82,89,0,1.34,ZA,1594954457,Misty Waves Boutique Hotel
5,Assiut,27.18,31.18,298.15,61,0,5.70,EG,1594954460,Florence Hotel Assiut
8,Bredasdorp,-34.53,20.04,280.15,87,0,3.10,ZA,1594954463,Bredasdorp Country Manor


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_df = [info_box_template.format(**row) for index, row in city_data.iterrows()]
locations = city_data[["Lat", "Lng"]]
cities = list(city_data["City"])


In [19]:
# Add marker layer ontop of heat map

symbol_layer = gmaps.marker_layer(
    locations,hover_text=cities,
    info_box_content=hotel_df,
    display_info_box=True
)


fig = gmaps.figure(center=(35,135),zoom_level=3)
fig.add_layer(symbol_layer)
fig.add_layer(heat_layer)


fig



Figure(layout=FigureLayout(height='420px'))